In [ ]:
# HANK-SAM Model Interactive Dashboard.
# Author: Alan Lujan <alujan@jhu.edu>

# This Voila dashboard allows interactive exploration of the HANK-SAM model's
# fiscal multipliers under different monetary and fiscal policy parameters.

# CRITICAL: Remove all whitespace above dashboard
from IPython.display import HTML, display


In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# Import required packages
import ipywidgets as widgets
from IPython.display import clear_output, display, HTML
from ipywidgets import HBox, Layout, VBox, ButtonStyle
import plotly.graph_objects as go
from plotly.graph_objs import FigureWidget
from plotly.subplots import make_subplots
import numpy as np
import asyncio
from tornado import ioloop  # Import tornado for proper event loop scheduling

# Import our refactored model module and branding
import hank_sam as hs
from branding.econ_ark_style import (
    ARK_BLUE, ARK_LIGHTBLUE, ARK_ORANGE, ARK_GREEN,
    ARK_SLATE_DK, ARK_SLATE_LT, ARK_GREY, ARK_GRID,
    ARK_PANEL, ARK_PANEL_LIGHT, ARK_GRID_SOFT, ARK_SPINE,
    ARK_TEXT, MATPLOTLIB_STYLE, DASHBOARD_CSS, HEADER_HTML, FAVICON_HTML, tidy_legend,
    DASH_STYLE
)

# Set custom page title for browser tab
display(HTML("<script>document.title = 'Econ-ARK HA Policy Dash'</script>"))

# Import constants
C_ss = 0.6910496136078721  # From hank_sam.py

# Global variables for auto-solve functionality
_pending_solve = False
_is_solving = False
_last_change_time = 0

# Configure Plotly template with Econ-ARK branding for sharp, modern plots
plotly_layout = dict(
    font=dict(family="system-ui, -apple-system, sans-serif", size=12, color=ARK_TEXT),
    paper_bgcolor='white',
    plot_bgcolor='white',
    margin = dict(l=60, r=30, t=35, b=40),
    hovermode='x unified',
    hoverlabel=dict(
        bgcolor='rgba(255, 255, 255, 0.95)',
        bordercolor='#d1d5db',
        font=dict(size=12, color=ARK_TEXT)
    ),
    xaxis=dict(
        gridcolor='#e5e7eb',
        linecolor='#d1d5db',
        showgrid=True,
        zeroline=True,
        zerolinecolor='#9ca3af',
        title_font=dict(size=13)
    ),
    yaxis=dict(
        gridcolor='#e5e7eb', 
        linecolor='#d1d5db',
        showgrid=True,
        zeroline=True,
        zerolinecolor='#9ca3af',
        title_font=dict(size=13)
    )
)



# Apply global dashboard styles with custom additions - FIX ALL CSS ISSUES
custom_css = DASH_STYLE

def create_heading(text, level=2, style_class=""):
    """Create a heading widget with consistent styling."""
    tag = f"h{level}"
    class_str = f"ark-h{level} {style_class}".strip()
    # Add plot-heading class for plot titles to reduce bottom margin
    if style_class == "lightblue":
        class_str += " plot-heading"
    return widgets.HTML(f"<{tag} class='{class_str}'>{text}</{tag}>")

# Create style for sliders - labels will be hidden since we'll add them above
style = {
    "description_width": "0px",  # Hide the default label
}
slider_layout = Layout(width="100%", margin="0", padding="0")

# Function to create a labeled slider with label above - FIX: Proper format strings
def create_labeled_slider(description, value, min_val, max_val, step, format_str=".2f", is_int=False):
    """Create a slider with inline label and value display."""
    
    # Create the appropriate slider type
    if is_int:
        slider = widgets.IntSlider(
            value=value,
            min=min_val,
            max=max_val,
            step=step,
            style={'description_width': '0px'},
            layout=widgets.Layout(width='auto', flex='1', margin='0.5rem 0.75rem'),
            continuous_update=False,
            readout=False,  # We'll create our own readout
        )
        # Format the value display
        value_text = str(value)
    else:
        # Determine format based on step size
        if step >= 0.1:
            actual_format = ".1f"
        elif step >= 0.01:
            actual_format = ".2f"
        else:
            actual_format = format_str
            
        slider = widgets.FloatSlider(
            value=value,
            min=min_val,
            max=max_val,
            step=step,
            style={'description_width': '0px'},
            layout=widgets.Layout(width='auto', flex='1', margin='0.5rem 0.75rem'),
            continuous_update=False,
            readout=False,  # We'll create our own readout
        )
        value_text = f"{value:{actual_format}}"
    
    # Create inline label with value
    label_html = widgets.HTML(
        value=f'<div class="inline-slider-label">{description} <span class="inline-slider-value">{value_text}</span></div>'
    )
    
    # Update label when slider changes
    def update_label(change):
        if is_int:
            new_value = str(change['new'])
        else:
            new_value = f"{change['new']:{actual_format}}"
        label_html.value = f'<div class="inline-slider-label">{description} <span class="inline-slider-value">{new_value}</span></div>'
    
    slider.observe(update_label, names='value')
    
    # Stack label and slider vertically but tightly
    container = VBox(
        [label_html, slider],
        layout=Layout(margin="0 0 0.25rem 0", padding="0", width="100%")
    )
    container.add_class("slider-container-inline")
    
    return container, slider

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
scenario_tabs = widgets.Tab(layout=Layout(margin="0", padding="0"))

# Store all widgets and preset buttons for each scenario
all_scenario_widgets = {}
all_preset_buttons = {}

# Store active presets for labeling - initialize with defaults using descriptive labels
active_presets = {0: 'Baseline', 1: 'Short tax-cut dur.', 2: 'High output target', 3: 'High inf. target'}


# Suppress tornado timer logging noise
import logging
logging.getLogger('tornado').setLevel(logging.ERROR)
logging.getLogger('tornado.application').setLevel(logging.ERROR)
logging.getLogger('tornado.access').setLevel(logging.ERROR)

# Auto-solve state management - FIXED by triggering button click instead of calling function directly
_auto_solve_enabled = True
_solve_scheduled = False  # Track if a solve is needed
_solve_timer = None
_is_auto_triggered = False  # Track if this is an auto-triggered solve

def schedule_auto_solve():
    """Schedule (or queue) an auto-solve."""
    global _solve_scheduled, _solve_timer, _is_auto_triggered
    
    # Always mark that a solve is desired
    _solve_scheduled = True
    
    # If a solve is already running, just queue the request;
    # the finally-block of update_plots() will pick it up.
    if _is_solving:
        progress_label.value = update_status("Queued...")  # Optional visual feedback
        return
    
    # If auto-solve is disabled, don't schedule
    if not _auto_solve_enabled:
        return
    
    # Cancel any existing scheduled solve
    if _solve_timer is not None:
        ioloop.IOLoop.current().remove_timeout(_solve_timer)
        _solve_timer = None
    
    # Schedule new solve after delay using tornado's event loop (works with Voila)
    # CRITICAL FIX: Trigger button click instead of calling update_plots directly
    def trigger_solve():
        if not _is_solving:
            import sys
            import os
            # Suppress output during trigger
            old_stdout = sys.stdout
            try:
                sys.stdout = open(os.devnull, 'w')
                # Mark this as an auto-triggered solve
                _is_auto_triggered = True
                # Programmatically click the button - this ensures proper output widget context
                run_button.click()
            finally:
                sys.stdout = old_stdout
    
    import sys
    import os
    old_stdout = sys.stdout
    try:
        sys.stdout = open(os.devnull, 'w')
        _solve_timer = ioloop.IOLoop.current().call_later(0.8, trigger_solve)
    finally:
        sys.stdout = old_stdout

# Function to create parameter widgets for a scenario with default preset
def create_scenario_widgets(scenario_num, default_preset=None):
    """Create a complete set of parameter widgets for one scenario."""
    
    # Define base default values
    default_values = {
        'phi_pi': 1.5,
        'phi_y': 0.0,
        'phi_b': 0.015,
        'ui_extension': 4,
        'tax_cut': 8
    }
    
    # Apply default preset values if specified
    if default_preset == 'tax_cut_dur':
        default_values['tax_cut'] = 1
    elif default_preset == 'high_output':
        default_values['phi_y'] = 0.95
    elif default_preset == 'high_inf':
        default_values['phi_pi'] = 2.0
    # baseline preset uses all default values
    
    # Create labeled sliders with appropriate default values - FIX: Use proper format strings
    phi_pi_container, phi_pi = create_labeled_slider(
        "Taylor rule inflation weight (φπ):", default_values['phi_pi'], 1.0, 3.0, 0.1, ".1f"  # Changed from .2f
    )
    
    phi_y_container, phi_y = create_labeled_slider(
        "Taylor rule output weight (φy):", default_values['phi_y'], 0.0, 1.0, 0.05, ".2f"  # Keep .2f for small increments
    )
    
    phi_b_container, phi_b = create_labeled_slider(
        "Fiscal adjustment (φb):", default_values['phi_b'], 0.0, 0.1, 0.005, ".3f"  # Keep .3f for very small values
    )
    
    ui_extension_container, ui_extension = create_labeled_slider(
        "UI extension (quarters):", default_values['ui_extension'], 1, 12, 1, is_int=True
    )
    
    tax_cut_container, tax_cut = create_labeled_slider(
        "Tax cut (quarters):", default_values['tax_cut'], 1, 16, 1, is_int=True
    )
    
    # Store widgets for this scenario
    scenario_widgets = {
        'phi_pi': phi_pi,
        'phi_y': phi_y,
        'phi_b': phi_b,
        'ui_extension': ui_extension,
        'tax_cut': tax_cut
    }
    
    # Create preset buttons using ipywidgets with descriptive labels
    preset_buttons = {}
    
    # Define preset values for comparison
    preset_values = {
        'baseline': {'tax_cut': 8, 'ui_extension': 4, 'phi_y': 0.0, 'phi_pi': 1.5, 'phi_b': 0.015},
        'tax_cut_dur': {'tax_cut': 1, 'ui_extension': 4, 'phi_y': 0.0, 'phi_pi': 1.5, 'phi_b': 0.015},
        'high_output': {'tax_cut': 8, 'ui_extension': 4, 'phi_y': 0.95, 'phi_pi': 1.5, 'phi_b': 0.015},
        'high_inf': {'tax_cut': 8, 'ui_extension': 4, 'phi_y': 0.0, 'phi_pi': 2.0, 'phi_b': 0.015}
    }
    
    # Function to check if current values match any preset
    def check_preset_match():
        current_values = {
            'tax_cut': scenario_widgets['tax_cut'].value,
            'ui_extension': scenario_widgets['ui_extension'].value,
            'phi_y': scenario_widgets['phi_y'].value,
            'phi_pi': scenario_widgets['phi_pi'].value,
            'phi_b': scenario_widgets['phi_b'].value
        }
        
        # Clear all active states for THIS scenario only
        for pb in preset_buttons.values():
            pb.remove_class('active')
        
        # Check if current values match any preset
        preset_matched = False
        for preset_type, preset_vals in preset_values.items():
            if all(abs(current_values[k] - preset_vals[k]) < 0.001 for k in preset_vals):
                if preset_type in preset_buttons:
                    preset_buttons[preset_type].add_class('active')
                    preset_matched = True
                    # Update active preset tracking with descriptive labels
                    preset_labels = {
                        'baseline': 'Baseline',
                        'tax_cut_dur': 'Short tax-cut dur.',
                        'high_output': 'High output target',
                        'high_inf': 'High inf. target'
                    }
                    active_presets[scenario_num - 1] = preset_labels[preset_type]
                break
        
        # If no preset matches, mark as custom
        if not preset_matched:
            active_presets[scenario_num - 1] = 'Custom'
    
    # Add value change handlers to all sliders for preset tracking AND auto-solve
    def on_slider_change(change):
        if change['name'] == 'value' and change['new'] != change['old']:
            check_preset_match()
            # Trigger auto-solve
            schedule_auto_solve()
    
    phi_pi.observe(on_slider_change, names='value')
    phi_y.observe(on_slider_change, names='value')
    phi_b.observe(on_slider_change, names='value')
    ui_extension.observe(on_slider_change, names='value')
    tax_cut.observe(on_slider_change, names='value')
    
    def create_preset_button(label, preset_type, width='78px', large=False):
        btn = widgets.Button(
            description=label,
            layout=widgets.Layout(width=width, height='24px', padding='0', margin='0 3px 0 0')
        )
        btn.add_class('preset-btn-large' if large else 'preset-btn')  # Use large class for longer text
        if preset_type == default_preset:     # mark default as selected
            btn.add_class('active')
        
        def on_click(b):
            # Apply preset values
            values = preset_values[preset_type]
            scenario_widgets['tax_cut'].value = values['tax_cut']
            scenario_widgets['ui_extension'].value = values['ui_extension']
            scenario_widgets['phi_y'].value = values['phi_y']
            scenario_widgets['phi_pi'].value = values['phi_pi']
            scenario_widgets['phi_b'].value = values['phi_b']
            
            # Clear all active states for THIS scenario panel only
            for pb in preset_buttons.values():   
                pb.remove_class('active')
            b.add_class('active')                # mark new selection
            
            # Update active preset tracking with descriptive labels
            preset_labels = {
                'baseline': 'Baseline',
                'tax_cut_dur': 'Short tax-cut dur.',
                'high_output': 'High output target',
                'high_inf': 'High inf. target'
            }
            active_presets[scenario_num - 1] = preset_labels[preset_type]
            
            # Trigger auto-solve when preset is clicked
            schedule_auto_solve()
        
        btn.on_click(on_click)
        return btn
        
    # Create all preset buttons with descriptive labels and appropriate widths - smaller fonts/widths
    preset_buttons['baseline'] = create_preset_button('Baseline', 'baseline', width='70px')
    preset_buttons['tax_cut_dur'] = create_preset_button('Short tax-cut dur.', 'tax_cut_dur', width='115px')
    preset_buttons['high_output'] = create_preset_button('High output target', 'high_output', width='115px')
    preset_buttons['high_inf'] = create_preset_button('High inf. target', 'high_inf', width='95px')
    
    # Store preset buttons for this scenario
    all_preset_buttons[scenario_num] = preset_buttons
    
    # Create preset buttons container with better spacing and centering - FIX: Add bottom padding
    preset_buttons_container = HBox(
        list(preset_buttons.values()),
        layout=Layout(
            margin='1rem 0 1.5rem 0',  # Much more space above and below
            padding='0',
            justify_content='center',
            align_items='center',
            width='100%'
        )
    )
    
    # Create layout for this scenario with tighter spacing
    policy_duration_group = VBox(
        [ui_extension_container, tax_cut_container],
        layout=Layout(
            padding='0.5rem',
            background='#ffffff',
            border='2px solid ' + ARK_BLUE,
            border_radius='8px',
            width='100%',
            margin='0 0 0.5rem 0'
        )
    )
    policy_duration_group.add_class("parameter-card")
    policy_duration_group.add_class("primary-controls")
    
    monetary_fiscal_group = VBox(
        [phi_pi_container, phi_y_container, phi_b_container],
        layout=Layout(
            padding='0.5rem',
            background='#f8fafc',
            border='1px solid #e2e8f0',
            border_radius='8px',
            width='100%',
            margin='0 0 0.5rem 0'
        )
    )
    monetary_fiscal_group.add_class("parameter-card")
    monetary_fiscal_group.add_class("secondary-controls")
    
    # Create section headings with smaller spacing and no top margin
    policy_duration_heading = widgets.HTML(f"""
    <h3 class="section-heading" style="margin: 0 0 0.3rem 0;">Policy Duration</h3>
    """)
    
    settings_heading = widgets.HTML("""
    <h3 class="section-heading" style="margin-bottom: 0.3rem;">Monetary and Fiscal Policy Settings</h3>
    """)
    
    
    # Create footnote text for policy details - small and subtle
    policy_footnote = widgets.HTML("""
    <div style='margin: 0.5rem 0 0 0; padding: 0.4rem 0.6rem; 
                background: #f9fafb; border-left: 2px solid #e5e7eb;
                border-radius: 2px;'>
        <p style='margin: 0; font-size: 0.65rem; color: #6b7280; line-height: 1.4;'>
            <em>Each simulation compares one policy to a no‑stimulus baseline under the same monetary‑policy rule. 
            Consumption multipliers are the ratio of cumulative consumption response to the cumulative fiscal stimulus. 
            Consumption response is the percent change in consumption relative to the no‑stimulus baseline.</em>
        </p>
    </div>
    """)
    
    # Combine into scenario panel with presets at the TOP and footnote at bottom
    scenario_panel = VBox(
        [
            preset_buttons_container,
            policy_duration_heading,
            policy_duration_group,
            settings_heading,
            monetary_fiscal_group,
            policy_footnote,  # Add footnote here
        ],
        layout=Layout(
            padding="0",
            width="100%",
            margin="0",
        )
    )
    
    # Return both the panel and the actual slider widgets
    return scenario_panel, scenario_widgets

# Create widgets for four scenarios with their default presets
scenario1_panel, scenario1_widgets = create_scenario_widgets(1, default_preset='baseline')     # Baseline
scenario2_panel, scenario2_widgets = create_scenario_widgets(2, default_preset='tax_cut_dur')  # Counterfactual 1
scenario3_panel, scenario3_widgets = create_scenario_widgets(3, default_preset='high_output')  # Counterfactual 2
scenario4_panel, scenario4_widgets = create_scenario_widgets(4, default_preset='high_inf')     # Counterfactual 3

# Store all widgets globally
all_scenario_widgets[1] = scenario1_widgets
all_scenario_widgets[2] = scenario2_widgets
all_scenario_widgets[3] = scenario3_widgets
all_scenario_widgets[4] = scenario4_widgets

# Add panels to tabs
scenario_tabs.children = [scenario1_panel, scenario2_panel, scenario3_panel, scenario4_panel]
scenario_tabs.set_title(0, 'Baseline')
scenario_tabs.set_title(1, 'Counterfactual 1')
scenario_tabs.set_title(2, 'Counterfactual 2')
scenario_tabs.set_title(3, 'Counterfactual 3')

# Create a status panel with button and progress label
run_button = widgets.Button(
    description="Simulate",
    layout=Layout(width="100%", height="36px"),
    style={"button_color": ARK_ORANGE, "font_weight": "600"},
)

# Create initial status HTML with Econ-ARK styling
initial_status_html = f"""
<div class="status-container" style="display: flex; align-items: center; justify-content: center; gap: 0.75em;">
    <div style="width: 8px; height: 8px; border-radius: 50%; 
                background: {ARK_GREY};">
    </div>
    <span style="color: {ARK_SLATE_DK}; font-size: 0.875rem;">Ready to run simulation</span>
</div>
"""

progress_label = widgets.HTML(
    value=initial_status_html,
    layout=Layout(
        width="100%", 
        margin="0.3em 0 0 0"
    )
)

# Create the status panel WITHOUT the "Simulation Control" heading
status_panel = widgets.VBox(
    [run_button, progress_label],
    layout=Layout(
        width="100%",
        padding="0",
        margin="0.5rem 0 0 0"
    )
)

# Create placeholder message for plots with WHITE background
placeholder_html = f"""
<div style="display:flex; flex-direction:column; align-items:center; justify-content:center; 
            background-color:white; border-radius:8px; padding:1.5em;
            height:250px;">
    <div style="font-size:1.1rem; color:{ARK_SLATE_DK}; margin-bottom:0.75em;">
        ⚡ Run Simulation to Generate Plots
    </div>
    <div style="font-size:0.9rem; color:{ARK_GREY}; text-align:center; max-width:300px;">
        Adjust parameters in all scenarios and click 'Simulate' to compare fiscal policy effects.
    </div>
</div>
"""

# Create FigureWidgets for direct plot updates (replaces Output widgets)
# This solves the async update issues in Voila
fig1_widget = FigureWidget()
fig2_widget = FigureWidget()

# Set initial empty state with placeholder message
def setup_empty_figure(fig_widget, title):
    fig_widget.data = []
    fig_widget.layout = go.Layout(
        height=400,  # Taller to use screen space  # Match the plot height
        margin=dict(l=60, r=30, t=85, b=50),
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(family="Inter, system-ui, -apple-system, sans-serif", size=12, color='#1a202c'),
        annotations=[
            dict(
                text="⚡ Run Simulation to Generate Plots",
                xref="paper", yref="paper",
                x=0.5, y=0.6,
                showarrow=False,
                font=dict(size=18, color='#1a202c'),
                xanchor='center', yanchor='middle'
            ),
            dict(
                text="Adjust parameters in all scenarios and click 'Simulate' to compare fiscal policy effects.",
                xref="paper", yref="paper",
                x=0.5, y=0.4,
                showarrow=False,
                font=dict(size=14, color='#6b7280'),
                xanchor='center', yanchor='middle'
            )
        ],
        xaxis=dict(visible=False),
        yaxis=dict(visible=False)
    )

# Initialize with placeholder
setup_empty_figure(fig1_widget, "Fiscal Multipliers")
setup_empty_figure(fig2_widget, "Consumption Response")

def update_status(msg, is_final=False, is_error=False):
    """Create status HTML with consistent styling."""
    global _is_auto_triggered
    
    # Add indicator if this is an auto-triggered solve
    if _is_auto_triggered and not is_final and not is_error:
        msg = "Auto-Solving..."
    
    if is_error:
        dot_color = ARK_ORANGE
        animate = ""
    elif is_final:
        dot_color = ARK_GREEN
        animate = ""
        msg = "Complete"
        _is_auto_triggered = False  # Reset the flag
    else:
        dot_color = ARK_ORANGE
        animate = "animation: pulse 1.2s ease-in-out infinite;"
        if not _is_auto_triggered:
            msg = "Solving..."
        
    return f"""
    <div style="display: flex; align-items: center; justify-content: center; gap: 0.8em;">
        <div style="width: 12px; height: 12px; border-radius: 50%; 
                    background: {dot_color}; {animate}">
        </div>
        <span style="color: {ARK_SLATE_DK}; font-weight: 500;">
            {msg}
        </span>
    </div>
    <style>
        @keyframes pulse {{
            0% {{ transform: scale(0.95); opacity: 0.7; }}
            50% {{ transform: scale(1.05); opacity: 1; }}
            100% {{ transform: scale(0.95); opacity: 0.7; }}
        }}
    </style>
    """

# Four-scenario PLOTLY plotting functions with Econ-ARK branding
def plot_scenario_comparison_multipliers_four_plotly(mult1, mult2, mult3, mult4, preset_labels=None):
    """Plot multipliers comparing four scenarios using Plotly."""
    
    # Colors and line styles for scenarios matching Econ-ARK branding
    colors = {'s1': ARK_BLUE, 's2': ARK_ORANGE, 's3': ARK_GREEN, 's4': '#ec4899'}
    dash_styles = {'s1': 'solid', 's2': 'dash', 's3': 'dot', 's4': 'dashdot'}
    
    # Labels with preset info if provided
    labels = {
        's1': f'Baseline ({preset_labels[0]})' if preset_labels and preset_labels.get(0) else 'Baseline',
        's2': f'CF1 ({preset_labels[1]})' if preset_labels and preset_labels.get(1) else 'CF1',
        's3': f'CF2 ({preset_labels[2]})' if preset_labels and preset_labels.get(2) else 'CF2',
        's4': f'CF3 ({preset_labels[3]})' if preset_labels and preset_labels.get(3) else 'CF3'
    }
    
    policies = ['Stimulus Check', 'UI Extension', 'Tax Cut']
    mult_keys = ['transfers', 'UI_extend', 'tax_cut']
    
    horizon_length = 20
    x_axis = np.arange(horizon_length) + 1
    
    # Determine common y-axis limits across all policies and scenarios
    all_max_values = []
    for key in mult_keys:
        all_max_values.extend([
            max(mult1[key][:horizon_length]),
            max(mult2[key][:horizon_length]),
            max(mult3[key][:horizon_length]),
            max(mult4[key][:horizon_length])
        ])
    y_max = max(all_max_values) * 1.2
    y_min = -0.2
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=policies,
        horizontal_spacing=0.06,  # Tighter spacing = wider plots
        vertical_spacing=0.1
    )
    
    # Plot each policy type
    for i, (policy, key) in enumerate(zip(policies, mult_keys)):
        col = i + 1
        
        # Add traces for all four scenarios
        for scenario_idx, (scenario_key, mult_data) in enumerate([
            ('s1', mult1), ('s2', mult2), ('s3', mult3), ('s4', mult4)
        ]):
            fig.add_trace(
                go.Scatter(
                    x=x_axis[:13],  # Limit to 12 quarters like the requirement
                    y=mult_data[key][:13],
                    mode='lines',
                    name=labels[scenario_key],
                    line=dict(
                        color=colors[scenario_key],
                        width=2.5,
                        dash=dash_styles[scenario_key]
                    ),
                    showlegend=(i == 0),  # Show legend only for first subplot
                    legendgroup=scenario_key,
                    hovertemplate='<b>%{fullData.name}</b><br>Q%{x}: %{y:.2f}<extra></extra>'
                ),
                row=1, col=col
            )
        
        # Update axes for this subplot
        fig.update_xaxes(
            title_text="Time (Quarters)" if i == 1 else "",
            range=[0.5, 12.5],
            gridcolor='#e5e7eb',
            linecolor='#d1d5db',
            showgrid=True,
            zeroline=False,
            row=1, col=col
        )
        
        fig.update_yaxes(
            title_text="Consumption Multiplier" if i == 0 else "",
            range=[y_min, y_max],
            gridcolor='#e5e7eb',
            linecolor='#d1d5db',
            showgrid=True,
            zeroline=True,
            zerolinecolor='#9ca3af',
            row=1, col=col
        )
    
    # Update layout with Econ-ARK branding
    fig.update_layout(
        height=400,  # Taller to use screen space
        margin = dict(l=60, r=30, t=35, b=35),  # Increased top margin from 50 to 80
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(
            family="Inter, system-ui, -apple-system, sans-serif",
            size=12,
            color=ARK_TEXT
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.13,  # Moved up more as requested
            xanchor="center",
            x=0.5,
            bgcolor="rgba(255, 255, 255, 0.9)",
            borderwidth=0,
            font=dict(size=12),  # Keep text size at 11
            itemsizing='constant',
            itemwidth=30,  # Minimum allowed width
            itemclick=False,  # Disable legend click
            itemdoubleclick=False  # Disable legend double-click
        ),

        hovermode='closest',
        hoverdistance=10,
        hoverlabel=dict(
            bgcolor='rgba(255, 255, 255, 0.95)',
            bordercolor='#d1d5db',
            font=dict(size=11, color=ARK_TEXT),
            align='left'
        )
    )
    

    # Optimize subplot titles for space
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(size=12, color=ARK_SLATE_DK, family="Inter, system-ui, sans-serif")
        annotation['yshift'] = -5  # Move titles closer to plots
    
    return fig

def plot_scenario_comparison_irfs_four_plotly(irfs1, irfs2, irfs3, irfs4, preset_labels=None):
    """Plot consumption IRFs comparing four scenarios using Plotly."""
    
    # Colors and line styles for scenarios
    colors = {'s1': ARK_BLUE, 's2': ARK_ORANGE, 's3': ARK_GREEN, 's4': '#ec4899'}
    dash_styles = {'s1': 'solid', 's2': 'dash', 's3': 'dot', 's4': 'dashdot'}
    
    # Labels with preset info if provided
    labels = {
        's1': f'Baseline ({preset_labels[0]})' if preset_labels and preset_labels.get(0) else 'Baseline',
        's2': f'CF1 ({preset_labels[1]})' if preset_labels and preset_labels.get(1) else 'CF1',
        's3': f'CF2 ({preset_labels[2]})' if preset_labels and preset_labels.get(2) else 'CF2',
        's4': f'CF3 ({preset_labels[3]})' if preset_labels and preset_labels.get(3) else 'CF3'
    }
    
    policies = ['Stimulus Check', 'UI Extension', 'Tax Cut']
    irf_keys = ['transfer', 'UI_extend', 'tau']
    
    Length = 12
    x_axis = np.arange(Length)
    
    # Determine common y-axis limits across all policies and scenarios
    all_max_values = []
    for key in irf_keys:
        all_max_values.extend([
            max(100 * irfs1[key]['C'][:Length] / C_ss),
            max(100 * irfs2[key]['C'][:Length] / C_ss),
            max(100 * irfs3[key]['C'][:Length] / C_ss),
            max(100 * irfs4[key]['C'][:Length] / C_ss)
        ])
    y_max = max(all_max_values) * 1.1
    y_min = -0.2
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=policies,
        horizontal_spacing=0.06,  # Tighter spacing = wider plots
        vertical_spacing=0.1
    )
    
    # Plot each policy type
    for i, (policy, key) in enumerate(zip(policies, irf_keys)):
        col = i + 1
        
        # Add traces for all four scenarios
        for scenario_idx, (scenario_key, irfs_data) in enumerate([
            ('s1', irfs1), ('s2', irfs2), ('s3', irfs3), ('s4', irfs4)
        ]):
            fig.add_trace(
                go.Scatter(
                    x=x_axis,
                    y=100 * irfs_data[key]['C'][:Length] / C_ss,
                    mode='lines',
                    name=labels[scenario_key],
                    line=dict(
                        color=colors[scenario_key],
                        width=2.5,
                        dash=dash_styles[scenario_key]
                    ),
                    showlegend=(i == 0),  # Show legend only for first subplot
                    legendgroup=scenario_key,
                    hovertemplate='<b>%{fullData.name}</b><br>Q%{x}: %{y:.2f}%<extra></extra>'
                ),
                row=1, col=col
            )
        
        # Update axes for this subplot
        fig.update_xaxes(
            title_text="Time (Quarters)" if i == 1 else "",
            gridcolor='#e5e7eb',
            linecolor='#d1d5db',
            showgrid=True,
            zeroline=False,
            row=1, col=col
        )
        
        fig.update_yaxes(
            title_text="Consumption Response (%)" if i == 0 else "",
            range=[y_min, y_max],
            gridcolor='#e5e7eb',
            linecolor='#d1d5db',
            showgrid=True,
            zeroline=True,
            zerolinecolor='#9ca3af',
            row=1, col=col
        )
    
    # Update layout with Econ-ARK branding
    fig.update_layout(
        height=400,  # Taller to use screen space
        margin=dict(l=60, r=30, t=85, b=40),  # Increased top margin from 50 to 80
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(
            family="Inter, system-ui, -apple-system, sans-serif",
            size=12,
            color=ARK_TEXT
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.13,  # Moved up more as requested
            xanchor="center",
            x=0.5,
            bgcolor="rgba(255, 255, 255, 0.9)",
            borderwidth=0,
            font=dict(size=12),  # Keep text size at 11
            itemsizing='constant',
            itemwidth=30,  # Minimum allowed width
            itemclick=False,  # Disable legend click
            itemdoubleclick=False  # Disable legend double-click
        ),
        hovermode='closest',
        hoverdistance=10,
        hoverlabel=dict(
            bgcolor='rgba(255, 255, 255, 0.95)',
            bordercolor='#d1d5db',
            font=dict(size=11, color=ARK_TEXT),
            align='left'
        )
    )
    

    # Optimize subplot titles for space
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(size=12, color=ARK_SLATE_DK, family="Inter, system-ui, sans-serif")
        annotation['yshift'] = -5  # Move titles closer to plots
    
    return fig

# Define update_plots function for four-scenario comparison - using PLOTLY
def update_plots(*args) -> None:
    """Run the four-scenario comparison with enhanced feedback."""
    global _is_solving, _solve_scheduled, _solve_timer, _is_auto_triggered
    
    # Clear the scheduled flag for this run
    _solve_scheduled = False
    
    # Cancel any pending timer
    if _solve_timer is not None:
        ioloop.IOLoop.current().remove_timeout(_solve_timer)
        _solve_timer = None
    
    # Prevent concurrent solves
    if _is_solving:
        return
    
    # Set solving flag
    _is_solving = True
    
    # Disable button and show solving status
    run_button.disabled = True
    progress_label.value = update_status("Solving Baseline...")

    try:
        # Get parameter values for all four scenarios
        scenario1_params = {
            "phi_pi": scenario1_widgets['phi_pi'].value,
            "phi_y": scenario1_widgets['phi_y'].value,
            "rho_r": 0.0,
            "kappa_p": 0.065,
            "phi_b": scenario1_widgets['phi_b'].value,
            "real_wage_rigidity": 0.95,
            "UI_extension_length": scenario1_widgets['ui_extension'].value,
            "tax_cut_length": scenario1_widgets['tax_cut'].value,
        }
        
        scenario2_params = {
            "phi_pi": scenario2_widgets['phi_pi'].value,
            "phi_y": scenario2_widgets['phi_y'].value,
            "rho_r": 0.0,
            "kappa_p": 0.065,
            "phi_b": scenario2_widgets['phi_b'].value,
            "real_wage_rigidity": 0.95,
            "UI_extension_length": scenario2_widgets['ui_extension'].value,
            "tax_cut_length": scenario2_widgets['tax_cut'].value,
        }
        
        scenario3_params = {
            "phi_pi": scenario3_widgets['phi_pi'].value,
            "phi_y": scenario3_widgets['phi_y'].value,
            "rho_r": 0.0,
            "kappa_p": 0.065,
            "phi_b": scenario3_widgets['phi_b'].value,
            "real_wage_rigidity": 0.95,
            "UI_extension_length": scenario3_widgets['ui_extension'].value,
            "tax_cut_length": scenario3_widgets['tax_cut'].value,
        }
        
        scenario4_params = {
            "phi_pi": scenario4_widgets['phi_pi'].value,
            "phi_y": scenario4_widgets['phi_y'].value,
            "rho_r": 0.0,
            "kappa_p": 0.065,
            "phi_b": scenario4_widgets['phi_b'].value,
            "real_wage_rigidity": 0.95,
            "UI_extension_length": scenario4_widgets['ui_extension'].value,
            "tax_cut_length": scenario4_widgets['tax_cut'].value,
        }

        # Run Baseline
        def update_computation_status(msg):
            progress_label.value = update_status("Solving Baseline...")
            
        results1 = hs.compute_fiscal_multipliers(
            status_callback=update_computation_status,
            **scenario1_params
        )
        
        # Run Counterfactual 1
        progress_label.value = update_status("Solving Counterfactual 1...")
        
        def update_computation_status2(msg):
            progress_label.value = update_status("Solving Counterfactual 1...")
            
        results2 = hs.compute_fiscal_multipliers(
            status_callback=update_computation_status2,
            **scenario2_params
        )
        
        # Run Counterfactual 2
        progress_label.value = update_status("Solving Counterfactual 2...")
        
        def update_computation_status3(msg):
            progress_label.value = update_status("Solving Counterfactual 2...")
            
        results3 = hs.compute_fiscal_multipliers(
            status_callback=update_computation_status3,
            **scenario3_params
        )
        
        # Run Counterfactual 3
        progress_label.value = update_status("Solving Counterfactual 3...")
        
        def update_computation_status4(msg):
            progress_label.value = update_status("Solving Counterfactual 3...")
            
        results4 = hs.compute_fiscal_multipliers(
            status_callback=update_computation_status4,
            **scenario4_params
        )

        # Extract results (only baseline Taylor rule)
        mult1 = {
            'transfers': results1["multipliers"]["transfers"],
            'UI_extend': results1["multipliers"]["UI_extend"],
            'tax_cut': results1["multipliers"]["tax_cut"]
        }
        
        mult2 = {
            'transfers': results2["multipliers"]["transfers"],
            'UI_extend': results2["multipliers"]["UI_extend"],
            'tax_cut': results2["multipliers"]["tax_cut"]
        }
        
        mult3 = {
            'transfers': results3["multipliers"]["transfers"],
            'UI_extend': results3["multipliers"]["UI_extend"],
            'tax_cut': results3["multipliers"]["tax_cut"]
        }
        
        mult4 = {
            'transfers': results4["multipliers"]["transfers"],
            'UI_extend': results4["multipliers"]["UI_extend"],
            'tax_cut': results4["multipliers"]["tax_cut"]
        }
        
        irfs1 = {
            'transfer': results1["irfs"]["transfer"],
            'UI_extend': results1["irfs"]["UI_extend"],
            'tau': results1["irfs"]["tau"]
        }
        
        irfs2 = {
            'transfer': results2["irfs"]["transfer"],
            'UI_extend': results2["irfs"]["UI_extend"],
            'tau': results2["irfs"]["tau"]
        }
        
        irfs3 = {
            'transfer': results3["irfs"]["transfer"],
            'UI_extend': results3["irfs"]["UI_extend"],
            'tau': results3["irfs"]["tau"]
        }
        
        irfs4 = {
            'transfer': results4["irfs"]["transfer"],
            'UI_extend': results4["irfs"]["UI_extend"],
            'tau': results4["irfs"]["tau"]
        }

        # Create PLOTLY figures and update FigureWidgets directly
        # This ensures reliable updates in Voila without async issues
        
        # Figure 1: Fiscal Multipliers using Plotly
        fig1 = plot_scenario_comparison_multipliers_four_plotly(
            mult1, mult2, mult3, mult4,
            preset_labels=active_presets
        )
        
        # Update FigureWidget directly - clear and add new data
        fig1_widget.data = []
        for trace in fig1.data:
            fig1_widget.add_trace(trace)
        fig1_widget.layout = fig1.layout
        
        # Figure 2: Consumption IRFs using Plotly
        fig2 = plot_scenario_comparison_irfs_four_plotly(
            irfs1, irfs2, irfs3, irfs4,
            preset_labels=active_presets
        )
        
        # Update FigureWidget directly
        fig2_widget.data = []
        for trace in fig2.data:
            fig2_widget.add_trace(trace)
        fig2_widget.layout = fig2.layout

        # Update summary statistics - comparing all four scenarios
        stimulus_mult1_1yr = mult1["transfers"][3]
        ui_mult1_1yr = mult1["UI_extend"][3]
        tax_mult1_1yr = mult1["tax_cut"][3]
        
        stimulus_mult2_1yr = mult2["transfers"][3]
        ui_mult2_1yr = mult2["UI_extend"][3]
        tax_mult2_1yr = mult2["tax_cut"][3]
        
        stimulus_mult3_1yr = mult3["transfers"][3]
        ui_mult3_1yr = mult3["UI_extend"][3]
        tax_mult3_1yr = mult3["tax_cut"][3]
        
        stimulus_mult4_1yr = mult4["transfers"][3]
        ui_mult4_1yr = mult4["UI_extend"][3]
        tax_mult4_1yr = mult4["tax_cut"][3]

        # HORIZONTAL LAYOUT for multipliers to save height
        summary_html = f"""
        <div style='display: grid; grid-template-columns: repeat(2, 1fr); gap: 0.3rem; 
                    max-width: 100%; margin: 0 auto;'>
            <div style='text-align: center;'>
                <h4 style='color: {ARK_SLATE_DK}; margin: 0 0 0.15rem 0; font-size: 0.62rem; 
                          font-weight: 600; text-transform: uppercase; letter-spacing: 0.02em;'>Baseline</h4>
                <div style='padding: 0.2rem 0.3rem; background-color: #fdfdfd; border-radius: 4px;
                            border: 1px solid #e2e8f0;'>
                    <div style='color: {ARK_TEXT}; font-size: 0.65rem; line-height: 1; 
                               font-family: Inter, system-ui, sans-serif; display: flex; justify-content: center; gap: 0.8em;'>
                        <span>Stim: <strong style='color: {ARK_SLATE_DK};'>{stimulus_mult1_1yr:.2f}</strong></span>
                        <span>UI: <strong style='color: {ARK_SLATE_DK};'>{ui_mult1_1yr:.2f}</strong></span>
                        <span>Tax: <strong style='color: {ARK_SLATE_DK};'>{tax_mult1_1yr:.2f}</strong></span>
                    </div>
                </div>
            </div>
            <div style='text-align: center;'>
                <h4 style='color: {ARK_SLATE_DK}; margin: 0 0 0.15rem 0; font-size: 0.62rem; 
                          font-weight: 600; text-transform: uppercase; letter-spacing: 0.02em;'>CF1</h4>
                <div style='padding: 0.2rem 0.3rem; background-color: #fdfdfd; border-radius: 4px;
                            border: 1px solid #e2e8f0;'>
                    <div style='color: {ARK_TEXT}; font-size: 0.65rem; line-height: 1; 
                               font-family: Inter, system-ui, sans-serif; display: flex; justify-content: center; gap: 0.8em;'>
                        <span>Stim: <strong style='color: {ARK_SLATE_DK};'>{stimulus_mult2_1yr:.2f}</strong></span>
                        <span>UI: <strong style='color: {ARK_SLATE_DK};'>{ui_mult2_1yr:.2f}</strong></span>
                        <span>Tax: <strong style='color: {ARK_SLATE_DK};'>{tax_mult2_1yr:.2f}</strong></span>
                    </div>
                </div>
            </div>
            <div style='text-align: center;'>
                <h4 style='color: {ARK_SLATE_DK}; margin: 0 0 0.15rem 0; font-size: 0.62rem; 
                          font-weight: 600; text-transform: uppercase; letter-spacing: 0.02em;'>CF2</h4>
                <div style='padding: 0.2rem 0.3rem; background-color: #fdfdfd; border-radius: 4px;
                            border: 1px solid #e2e8f0;'>
                    <div style='color: {ARK_TEXT}; font-size: 0.65rem; line-height: 1; 
                               font-family: Inter, system-ui, sans-serif; display: flex; justify-content: center; gap: 0.8em;'>
                        <span>Stim: <strong style='color: {ARK_SLATE_DK};'>{stimulus_mult3_1yr:.2f}</strong></span>
                        <span>UI: <strong style='color: {ARK_SLATE_DK};'>{ui_mult3_1yr:.2f}</strong></span>
                        <span>Tax: <strong style='color: {ARK_SLATE_DK};'>{tax_mult3_1yr:.2f}</strong></span>
                    </div>
                </div>
            </div>
            <div style='text-align: center;'>
                <h4 style='color: {ARK_SLATE_DK}; margin: 0 0 0.15rem 0; font-size: 0.62rem; 
                          font-weight: 600; text-transform: uppercase; letter-spacing: 0.02em;'>CF3</h4>
                <div style='padding: 0.2rem 0.3rem; background-color: #fdfdfd; border-radius: 4px;
                            border: 1px solid #e2e8f0;'>
                    <div style='color: {ARK_TEXT}; font-size: 0.65rem; line-height: 1; 
                               font-family: Inter, system-ui, sans-serif; display: flex; justify-content: center; gap: 0.8em;'>
                        <span>Stim: <strong style='color: {ARK_SLATE_DK};'>{stimulus_mult4_1yr:.2f}</strong></span>
                        <span>UI: <strong style='color: {ARK_SLATE_DK};'>{ui_mult4_1yr:.2f}</strong></span>
                        <span>Tax: <strong style='color: {ARK_SLATE_DK};'>{tax_mult4_1yr:.2f}</strong></span>
                    </div>
                </div>
            </div>
        </div>
        """

        # Update the summary section
        summary_row.children[0].children[1].value = summary_html

        # Show completion status
        progress_label.value = update_status("Complete", is_final=True)
        run_button.disabled = False
        
    except Exception as e:
        # Show error status
        progress_label.value = update_status(f"Error: {str(e)}", is_error=True)
        run_button.disabled = False
        _is_auto_triggered = False  # Reset flag on error
        # Reset figures to empty state on error
        setup_empty_figure(fig1_widget, "Fiscal Multipliers")
        setup_empty_figure(fig2_widget, "Consumption Response")
    finally:
        # CRITICAL: Reset solving flag at the end
        _is_solving = False
        
        # Check if there was a pending solve request while we were solving
        if _solve_scheduled:
            schedule_auto_solve()

# Connect button to update function
run_button.on_click(update_plots)

# Trigger initial auto-solve after a short delay by clicking the button programmatically
def trigger_initial_solve():
    """Trigger initial solve after dashboard loads."""
    global _is_auto_triggered
    import sys
    import os
    # Suppress the timer handle output
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    try:
        # Redirect output to devnull during the trigger
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')
        _is_auto_triggered = True
        run_button.click()
    finally:
        # Restore output
        sys.stdout = old_stdout
        sys.stderr = old_stderr

# Schedule the initial solve using tornado's event loop (works with Voila)
import sys
import os
# Suppress the timer handle output during scheduling
old_stdout = sys.stdout
try:
    sys.stdout = open(os.devnull, 'w')
    ioloop.IOLoop.current().call_later(0.5, trigger_initial_solve)
finally:
    sys.stdout = old_stdout

In [ ]:
# Remove ALL whitespace above header
display(HTML('''<style>
    /* AGGRESSIVE WHITESPACE REMOVAL */
    html, body {
        margin: 0 !important;
        padding: 0 !important;
    }
             
    /* Put this with the other AGGRESSIVE WHITESPACE rules */
    .ark-h2, h2 {
        margin-top: 0 !important;
        margin-bottom: 0.25rem !important;   /* tweak as desired */
    }

    
    /* Target all containers */
    #voila, #voila-container, .voila, .voila-container,
    #notebook, #notebook-container, .notebook, .notebook-container,
    .container, .container-fluid, .jp-Notebook, .jp-NotebookPanel {
        margin-top: 0 !important;
        padding-top: 0 !important;
    }
    
    /* Target cells */
    .jp-Cell, .jp-CodeCell, .jp-MarkdownCell, .cell {
        margin-top: 0 !important;
        padding-top: 0 !important;
    }
    
    /* First child rules */
    body > *:first-child {
        margin-top: 0 !important;
        padding-top: 0 !important;
    }
    
    /* The header itself - absolute positioning */
    .ark-header {
        position: absolute !important;
        top: 0 !important;
        left: 0 !important;
        right: 0 !important;
        margin: 0 !important;
        z-index: 9999 !important;
    }
    
    /* Main content spacing to account for header */
    .ark-header ~ * {
        margin-top: 60px !important;
    }
    
    /* Output and widget areas */
    .jp-OutputArea, .output_wrapper, .output,
    .widget-area, .widget-subarea, .widget-box {
        margin-top: 0 !important;
        padding-top: 0 !important;
    }
    
    /* First output cell */
    .jp-Cell:first-child .jp-OutputArea {
        margin-top: -10px !important;
        padding-top:  !important;
    }
</style>'''))

# ═════════════════════════════════════════════════════════════════════════════
# SIDEBAR - professional layout with scenario tabs
options_panel = VBox(
    [
        scenario_tabs,
        status_panel
    ],
    layout=Layout(
        padding="0.5rem",
        margin="0.5rem 0 0 0",
        width="100%",
        height="auto",
        min_height="450px",
        overflow_y="visible", 
        overflow_x="hidden",
        display="flex",
        flex_direction="column",
        justify_content="space-between",  # Distribute space evenly
    ),
)
options_panel.add_class("sidebar-container")

# MAIN CONTENT - Two figure panels with fixed layout
fig1_panel = VBox(
    [
        create_heading("Fiscal Multipliers", 2, "lightblue"),
        fig1_widget,  # Using FigureWidget for reliable updates
    ],
    layout=Layout(
        border="none",
        margin="0 -1em -1em 0",
        padding="0",
        width="100%",
        height="auto",
        min_height="420px",  # Match taller plots
        max_height="480px",  # Allow taller plots
        overflow="hidden",
    ),
)

fig2_panel = VBox(
    [
        create_heading("Consumption Response", 2, "lightblue"),
        fig2_widget,  # Using FigureWidget for reliable updates
    ],
    layout=Layout(
        border="none",
        padding="0",
        margin="0",
        width="100%",
        height="auto",
        min_height="420px",  # Match taller plots
        max_height="480px",  # Allow taller plots
        overflow="hidden",
    ),
)

# Create the key insights widget with improved bullet styling and updated content
key_insights_content = widgets.HTML("""
<div style='margin: 0; padding: 1rem; background-color: white; 
            border-radius: 8px; height: 100%; box-sizing: border-box;'>
    <ul class='key-insights-list' style='font-size: 1.1rem !important; line-height: 1.6;'>
        <li>UI extensions consistently deliver the largest multipliers because they channel funds to unemployed households with the highest marginal propensity to consume (MPC).</li>
        <li>The fiscal multiplier of the tax cut is small because much of the tax cuts go to wealthier individuals with a low MPC.</li>
        <li>A one quarter tax‑cut yields a similar multiplier to a stimulus check of $1,200.</li>
    </ul>
</div>
  """)

# Create summary statistics section for first row (will be populated by simulation results)
summary_stats_box = VBox(
    [
        create_heading("Average Multipliers (1-Year Horizon)", 2, "lightblue"),
        widgets.HTML(
            "<div id='summary-stats' style='margin: 0.5em 0 0 0; padding: 1.5em; "
            "background-color: white; border-radius: 8px; font-size: 1rem; text-align: center;' class='ark-label'>"
            "Run simulation to compare scenarios...</div>"
        ),
    ],
    layout=Layout(
        width="32%",  # Narrower width
        padding="0",
        margin="0 0 0 0",
        height="auto",
        min_height="100px",  # Reduced height
    ),
)

# Create key insights box with heading to match Key Multipliers
key_insights_box = VBox(
    [
        create_heading("Key Insights", 2, "green"),
        key_insights_content,
    ],
    layout=Layout(
        width="68%",  # More space for insights
        padding="0",
        margin="0",
        height="auto",
        min_height="100px",
    ),
)

# Create the summary row with both stats and key insights
summary_row = HBox(
    [summary_stats_box, key_insights_box],
    layout=Layout(
        width="100%",
        padding="0",
        margin="-0.5rem 0 -0.5em 0",  # Reduced bottom margin
        height="auto",
        align_items="stretch",
    ),
)

# Create introduction section with updated text
# Create introduction section with updated text
intro_section = VBox(
    [
        widgets.HTML("""
        <h1 class='ark-h1-pink'>Fiscal Interventions with Heterogeneous Consumers</h1>
        """),
        widgets.HTML("""
        <div style='margin: 0.3em 0 0.5em 0; padding: 0;'>
            <p style='margin: 0; line-height: 1.3; color: #2d3748; font-size: 1rem; 
                      text-align: justify; max-width: none;'>
                This dashboard quantifies the macroeconomic impact of three fiscal‑stimulus interventions by aggregating the consumption response across the income and wealth distribution. For each scenario, the dashboard solves the distributional steady state (without the intervention) in a Heterogeneous‑Agent New‑Keynesian (HANK) model with search‑and‑matching frictions. 
                Intervention dynamics are then simulated using the <a href='https://docs.econ-ark.org/examples/ConsNewKeynesianModel/HANKFiscal_example.html' style='color: #005b8f; text-decoration: none; font-weight: 500;' target='_blank'>sequence space jacobian (SSJ) method implemented in the HARK toolkit</a>.
            </p>
        </div>
        """),
        widgets.HTML("""
        <div style='margin: 0.5em 0 0.8em 0; padding: 0;'>
            <h3 style='font-size: 1rem; font-weight: 600; color: #1a202c; 
                       margin: 0 0 0.4em 0; letter-spacing: -0.01em; border-bottom: 1px solid #e2e8f0; 
                       padding-bottom: 0.2em;'>
                Interventions Simulated
            </h3>
            <ol style='margin: 0 0 0.5em 0; padding-left: 1.5em; line-height: 1.3; color: #4a5568; 
                       font-size: 0.95rem; list-style: decimal;'>
                <li style='margin-bottom: 0.3em;'>
                    <strong style='color: #2d3748; font-weight: 600;'>Stimulus Check</strong> – $1,200 one-off transfer. 
                </li>
                <li style='margin-bottom: 0.3em;'>
                    <strong style='color: #2d3748; font-weight: 600;'>UI Extension</strong> – Unemployment‑benefit horizon doubled from 6 to 12 months (for stimulus policy duration).
                </li>
                <li style='margin-bottom: 0;'>
                    <strong style='color: #2d3748; font-weight: 600;'>Tax cut</strong> - 2 percent take‑home pay increase (for stimulus policy duration).
                </li>
            </ol>
        </div>
        """),
    ],
    layout=Layout(
        width="100%",
        padding="0",
        margin="0 0 1rem 0",  # More space before settings
        height="auto",
        overflow="visible",
    ),
)
left_panel = VBox(
    [intro_section, options_panel],
    layout=Layout(
        width="30%",
        height="auto",  # Auto height
        min_height="600px",  # Minimum height for content
        display="flex",
        flex_direction="column",
        padding="0.75rem",
        background_color="#f8fafc",
        justify_content="flex-start",
    ),
)

# Right panel with summary row and plots (removed the general plot note)
right_panel = VBox(
    [summary_row, fig1_panel, fig2_panel],
    layout=Layout(
        width="70%",
        height="auto",
        min_height="450px",  # Fit better on 13-inch
        padding="0em 0em",
        background_color="white",
        overflow="visible",
        display="flex",
        flex_direction="column",
        gap="-10em",  # Even smaller gap
        justify_content="flex-start",
    ),
)

# Split horizontally: Options left (30%) -> Figures right (70%)
main_content = HBox(
    [left_panel, right_panel],
    layout=Layout(
        width="100%",
        height="auto",
        min_height="450x",  # Fit better on 13-inch
        overflow="visible",
        margin="0",
        padding="0",
        align_items="stretch",
    ),
)

# Add the header to the top of the dashboard
header_widget = widgets.HTML(HEADER_HTML)
favicon_widget = widgets.HTML(FAVICON_HTML)

# Inject the custom CSS along with the dashboard
css_widget = widgets.HTML(custom_css)

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════

# Add footer banner
footer = widgets.HTML("""
<div style='background:#f3f4f6; padding:20px 0; text-align:center; 
            font-size:0.85rem; color:#4b5563; margin-top:-1rem;
            border-top: 1px solid #e5e7eb;'>
    <div style='font-size:0.8rem; color:#6b7280;'>
        © 2025 Econ-ARK · Dashboard powered by Plotly FigureWidget & Voila
    </div>
</div>
""")

# Build the complete dashboard structure
dashboard = VBox(
    [css_widget, favicon_widget, header_widget, main_content, footer],
    layout=Layout(
        width="100%",
        height="auto",
        overflow="visible",
        margin="0",
        padding="0",
    ),
)

# Display dashboard
dashboard